In [1]:
import geopandas as gpd
from datetime import datetime
import shapely
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
#import seaborn as sns
from geopandas.tools import geocode
import folium

In [2]:
#!pip install pyshp
#!pip install geopandas
#!pip install folium
#!pip install geopy

In [3]:
def city_name(str):
    str = str[:str.find(', ')]
    k = str.find('(')
    if k != -1:
        str = str[:k]
    return str

def index_by_name(str):
    return cities[cities['Name'] == str].index.to_list()[0]

def name_by_index(i):
    return cities.iat[i, 0]

def temp_by_day(cit, dat):
    cityid = index_by_name(cit)
    ret = tempC.loc[(tempC['CityId'] == cityid) & (tempC['Date'] == dat),['Day temperature','Evening temperature']] 
    return ret

In [4]:
cities = pd.read_csv('data/cities.csv', sep=';', index_col = 0)
tempC = pd.read_csv('data/temp.csv', sep=';', index_col = 'Id', parse_dates=['Date']) # Temp in Celsiul

# Отбрасываем от названия города посторонний текст
for x in cities.index:
    cities.iat[x, 0] = city_name(cities.iat[x, 0])

In [5]:
tmp = cities.values.tolist()
df = pd.DataFrame({'cityname' : tmp})

def color_by_temp(val):
    colors = ['darkblue', 'blue', 'lightblue', 'pink', 'lightred', 'red',  'darkred']
    t_min = -30.0
    t_max = 30.0
    if val < t_min:
        val = t_min
    elif val > t_max:
        val = t_max
    i = int((val + t_max)/8.57)
    return colors[i]

def custom_geocoder(address):
    dataframe = geocode(address , provider="nominatim", user_agent = 'my_request')
    point = dataframe.geometry.iloc[0]
    return pd.Series({'Latitude': point.y, 'Longitude': point.x})
 
#Applying function to the dataframe
df[['latitude', 'longitude']] =  df.cityname.apply(lambda x: custom_geocoder(x))
 
# Make sure you run the above code first to plot the results
mapit = folium.Map(location=[53.198627, 50.113987], zoom_start = 2)
 
mydate = datetime.strptime('2023-02-23','%Y-%m-%d')
#for lat, lon in zip(df.latitude , df.longitude):
for index, row in df.iterrows():
    vcity, lat, lon = row['cityname'][0], row['latitude'], row['longitude']
    t1 = temp_by_day(vcity, mydate).iat[0,0]
    t2 = temp_by_day(vcity, mydate).iat[0,1]
    hcol = 'pink'
    if t1 != t1:
        st1 = "—"
    else:
        st1 = str(t1) + '°C'
        hcol = color_by_temp(t1)
        #print(hcol)
    if t2 != t2:
        st2 = "—"
    else:
        st2 = str(t2) + '°C'
    html = '''{city}<br>Днём: {t1}<br>Вечером: {t2}'''.format(city = vcity, t1 = st1, t2 = st2)
    iframe = folium.IFrame(html, width = 200, height = 80)
    popup = folium.Popup(iframe,  max_width = 200)
    folium.Marker( location=[ lat,lon ], icon = folium.Icon(color=hcol, icon_color='white', icon = 'cloud'), radius = 8, popup = popup ).add_to( mapit )
 
#map_df.plot(color='red', alpha=0.4)     
mapit

In [6]:
mapit.save("weather_data.html")